# Библиотеки Python для Data Science: Numpy, Matplotlib, Scikit-learn
# Курсовой проект

In [1483]:
import pandas as pd
import datetime
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

### 1 Чтение данных

In [1484]:
train_data = pd.read_csv("train.csv")
target_data = pd.read_csv("test.csv")
X = train_data.drop("Price", axis = 1)
y = train_data["Price"]
X_target = target_data

### 1.1 Очистка данных

In [1485]:
class FeatureImputer:
    
    def __init__(self):
        
        self.medians = None
        self.dist_hc1_med = None
        self.dist_hc1_median = None
        self.current_year = None
        self.dist_date_med = None
        self.rooms_lsquare_mean = None
        self.rooms_square_mean = None
    
    def fit(self, X):

        #Вычислим медианы
        self.medians = X.median()
        
        #Вычислим медиану Healthcare_1 для районов, если не определено - заменяем медианой по районам
        self.dist_hc1_med = X.groupby(['DistrictId'])[['Healthcare_1']].median()
        self.dist_hc1_median = self.dist_hc1_med['Healthcare_1'].median()
        self.dist_hc1_med['Healthcare_1'] = self.dist_hc1_med['Healthcare_1'].fillna(self.dist_hc1_median)
        
        #Определяем текущий год и медиану года дома по району
        self.current_year = datetime.datetime.now().year
        self.dist_date_med = X.groupby(['DistrictId'])[['HouseYear']].median()
        
        #Определяем среднее отношение жилой площади к количеству комнат
        self.rooms_lsquare_mean = (X.loc[X['Rooms']!=0,'LifeSquare']/X.loc[X['Rooms']!=0,'Rooms']).mean()    
        
        #Определим среднюю площадь для 1,2,3-х комнатных квартир площадью меньше 150
        data_roms_square = X.loc[(X['Square'] < 150) & (X['Rooms'] > 0) & (X['Rooms'] < 4)]
        self.rooms_square_mean = data_roms_square.groupby(['Rooms'])[['Square']].mean()
        
    def transform(self, X):
        
        #Удаляем неинформативные признаки
        X = X.drop(['Shops_2', 'Ecology_2', 'Ecology_3'], axis = 1)
        
        #Если дата больше текущей - заменяем ее медианой по району
        for dist_ind in self.dist_date_med.index:
            X.loc[(X['HouseYear'] > self.current_year) & (X['DistrictId'] == dist_ind),'HouseYear'] = self.dist_date_med.loc[dist_ind,'HouseYear']
        
        #Если этаж равен 0, то заменяем его 1
        #Если этажность дома равна 0, то приравниваем этажность этажу плюс 6 (коэффициент определен по датасету)
        #Если этажность меньше этажа, то меняем их местами
        #Если этажность дома больше 50, то заменяем медианной этажностью
        X.loc[X['Floor'] == 0, "Floor"] = 1
        X.loc[X['HouseFloor'] == 0, 'HouseFloor'] = X.loc[X['HouseFloor'] == 0, 'Floor'] + 6
        ind = X.loc[X['HouseFloor'] < X['Floor']].index
        X.loc[ind, 'HouseFloor'], X.loc[ind, 'Floor'] = X.loc[ind, 'Floor'], X.loc[ind, 'HouseFloor']
        X.loc[X['HouseFloor'] > 50, 'HouseFloor'] = self.medians['HouseFloor']
        
        #Заполняем Healthcare_1 средними значениями по району
        #Если района не было в тренировочном датасете, заменяем Healthcare_1 вычисленной медианой
        for dist_ind in X['DistrictId'].unique():
            if dist_ind in self.dist_hc1_med.index:
                X.loc[(pd.isna(X['Healthcare_1'])) & (X['DistrictId'] == dist_ind),'Healthcare_1'] = self.dist_hc1_med.loc[dist_ind,'Healthcare_1']
            else:
                X.loc[(pd.isna(X['Healthcare_1'])) & (X['DistrictId'] == dist_ind),'Healthcare_1'] = self.dist_hc1_median
        
        #Если жилая площадь больше общей или неопределена, то определяем как разницу общей площади и площади кухни
        #минус 5 (коэффициент определен по датасету). Если жилая площадь отрицательная - меняем местами с площадью кухни
        ind = X.loc[(X["LifeSquare"] > X["Square"]) | (pd.isna(X["LifeSquare"]))].index
        X.loc[ind, "LifeSquare"] = X.loc[ind, "Square"] - X.loc[ind, "KitchenSquare"] - 5
        ind = X.loc[(X["LifeSquare"] < 0)].index
        X.loc[ind, "LifeSquare"], X.loc[ind, "KitchenSquare"] = X.loc[ind, "KitchenSquare"], X.loc[ind, "LifeSquare"]
        
        #Если количество комнат равно 0, то заменяем 1
        #Если количество комнат больше 5 или не определено, то расчитываем его относительно жилой площади
        X.loc[X['Rooms'] == 0, 'Rooms'] = 1
        ind = X.loc[((X['Rooms'] > 5) | (pd.isna(X['Rooms'])))].index
        X.loc[ind, 'Rooms'] = round((X.loc[ind, 'LifeSquare'])/self.rooms_lsquare_mean)
        
        #Вычислим площадь квартиры относительно количества ее комнат
        for i in (1,2,3):
            X.loc[(X['Square'] > 150) & (X['Rooms'] == i),'Square'] = self.rooms_square_mean.loc[i,'Square']
            
        #Если площадь кухни больше чем разница общей и жилой площадей или меньше 0, то приравниваем к медиане площади кухни
        X.loc[((X["KitchenSquare"] > (X["Square"] - X["LifeSquare"])) | (X["KitchenSquare"] <= 0)), "KitchenSquare"] = self.medians['KitchenSquare']
        
        #Если жилая площадь оказалась меньше кухни - меняем местами
        ind = X.loc[(X["LifeSquare"] < X["KitchenSquare"])].index
        X.loc[ind, "LifeSquare"], X.loc[ind, "KitchenSquare"] = X.loc[ind, "KitchenSquare"], X.loc[ind, "LifeSquare"]
        
        #Если общая площадь оказалась меньше жилой - меняем местами
        ind = X.loc[(X["Square"] < X["LifeSquare"])].index
        X.loc[ind, 'LifeSquare'] = X.loc[ind, 'Square']
        
        #Если жилая площадь оказалась меньше 10, приравниваем к общей площади минус 10
        ind = X.loc[(X["LifeSquare"] < 10)].index
        X.loc[ind, 'LifeSquare'] = X.loc[ind, 'Square'] - 10
        
        #Если при площади квартиры больше 50 площадь кухни оказалась меньше 2 - заменяем медианой
        ind = X.loc[(X["KitchenSquare"] < 2) & (X["Square"] > 50)].index
        X.loc[ind, 'KitchenSquare'] = self.medians['KitchenSquare']
        
        return X

### 1.2 Feature engineering

In [1486]:
class FeatureGenetator():
    
    def __init__(self):
        
        self.DistrictId_counts = None
        self.binary_to_numbers = None
        self.med_price_by_district = None
        self.med_price_by_floor_year = None
        self.current_year = None
        self.stat = None
        self.mean_price_per_square = None
        
    def fit(self, X, y=None):
        
        #Определим количества домов в районе, оставим только районы, где домов больше 50
        self.current_year = datetime.datetime.now().year
        district = X['DistrictId'].value_counts()
        district = district[district > 50]
        self.dist_popular = dict(district)
        
        #Вычислим среднюю стоимость квадратного метра по району
        X = X.join(y)
        X['Mean_price_per_sq'] = X['Price']/X['Square']
        self.dist_mean_price_per_square = X.groupby("DistrictId",as_index=False)[['Mean_price_per_sq']].mean().rename(columns={'Mean_price_per_sq':'Mean_price_per_square'})
        X = X.merge(self.dist_mean_price_per_square, on="DistrictId", how='left')
        self.mean_price_per_square = X['Mean_price_per_square'].mean()
        X.drop(['Mean_price_per_square', 'Mean_price_per_sq'], axis = 1)
        
        #Вычислим среднюю площадь по району и количеству комнат
        self.dist_rooms_mean_square_per_rooms = X.groupby(['DistrictId', 'Rooms'],as_index=False)[['Square']].mean().rename(columns={'Square':'Mean_square_per_rooms'})
        X = X.merge(self.dist_rooms_mean_square_per_rooms, on=["DistrictId", 'Rooms'], how='left')
        self.mean_squre_per_rooms = X['Mean_square_per_rooms'].mean()
        X.drop('Mean_square_per_rooms', axis = 1)
        
        #Вычислим среднюю стоимость для количества комнат
        self.rooms_ean_price_per_rooms = X.groupby('Rooms',as_index=False)[['Price']].mean().rename(columns={'Price':'Mean_price_per_rooms'})
        X = X.merge(self.rooms_ean_price_per_rooms, on='Rooms', how='left')
        self.mean_price_per_rooms = X['Mean_price_per_rooms'].mean()
        
    def transform(self, X):
        
        #Добавим признак популярности района
        X['DistrictId_count'] = X['DistrictId'].map(self.dist_popular)        
        X['DistrictId_count'].fillna(10, inplace=True)
        
        #Добавим признак возрастной категории дома
        X['Year_category'] = round((self.current_year - X['HouseYear'])/10)
        
        #Добавим признак - средняя стоимость квадратного метра в районе
        X = X.merge(self.dist_mean_price_per_square, on="DistrictId", how='left')
        X['Mean_price_per_square'] = X['Mean_price_per_square'].fillna(self.mean_price_per_square)
        
        #Добавим признак - средняя площадь для количества комнат в районе
        X = X.merge(self.dist_rooms_mean_square_per_rooms, on=["DistrictId", 'Rooms'], how='left')
        X['Mean_square_per_rooms'] = X['Mean_square_per_rooms'].fillna(self.mean_squre_per_rooms)
        
        #Добавим признак - средняя стоимость для количества комнат
        X = X.merge(self.rooms_ean_price_per_rooms, on='Rooms', how='left')
        X['Mean_price_per_rooms'] = X['Mean_price_per_rooms'].fillna(self.mean_price_per_rooms)
        
        return X

### 2 Обработка данных

In [1487]:
imputer = FeatureImputer()

imputer.fit(X)

X = imputer.transform(X)
X_target = imputer.transform(X_target)


features = FeatureGenetator()

features.fit(X, y)

X = features.transform(X)
X_target = features.transform(X_target)

### 3 Построение модели

In [1488]:
model1 = CatBoostRegressor(cat_features = ['DistrictId'])
model2 = RandomForestRegressor(n_estimators=1442, max_depth=18, random_state=42, max_features=7)
model = VotingRegressor([('model1', model1), ('model2', model2)])
scores = cross_val_score(model, X, y, cv=5)

model.fit(X, y)
y_pred = model.predict(X)

Learning rate set to 0.056513
0:	learn: 89564.6723158	total: 13.8ms	remaining: 13.8s
1:	learn: 86647.0865148	total: 19.9ms	remaining: 9.93s
2:	learn: 83975.3674110	total: 34ms	remaining: 11.3s
3:	learn: 81387.2746162	total: 48.6ms	remaining: 12.1s
4:	learn: 78958.2693263	total: 64.3ms	remaining: 12.8s
5:	learn: 76739.2708411	total: 74.7ms	remaining: 12.4s
6:	learn: 74608.7222026	total: 87.9ms	remaining: 12.5s
7:	learn: 72750.0339621	total: 99.1ms	remaining: 12.3s
8:	learn: 70906.2891524	total: 114ms	remaining: 12.6s
9:	learn: 69231.9886390	total: 128ms	remaining: 12.7s
10:	learn: 67706.6377240	total: 137ms	remaining: 12.3s
11:	learn: 66303.3636421	total: 148ms	remaining: 12.2s
12:	learn: 64987.1090761	total: 163ms	remaining: 12.4s
13:	learn: 63767.7517725	total: 178ms	remaining: 12.5s
14:	learn: 62633.6351440	total: 197ms	remaining: 12.9s
15:	learn: 61560.8167102	total: 213ms	remaining: 13.1s
16:	learn: 60531.0286884	total: 228ms	remaining: 13.2s
17:	learn: 59609.4003540	total: 244ms	r

153:	learn: 41759.9445827	total: 1.64s	remaining: 9.03s
154:	learn: 41734.6832083	total: 1.68s	remaining: 9.14s
155:	learn: 41706.1134094	total: 1.69s	remaining: 9.13s
156:	learn: 41686.4850346	total: 1.7s	remaining: 9.14s
157:	learn: 41660.3006321	total: 1.72s	remaining: 9.15s
158:	learn: 41606.3926580	total: 1.72s	remaining: 9.12s
159:	learn: 41554.2150224	total: 1.74s	remaining: 9.14s
160:	learn: 41512.6794518	total: 1.75s	remaining: 9.14s
161:	learn: 41495.8098691	total: 1.76s	remaining: 9.11s
162:	learn: 41466.3289780	total: 1.77s	remaining: 9.07s
163:	learn: 41464.0123353	total: 1.77s	remaining: 9.05s
164:	learn: 41435.1362227	total: 1.78s	remaining: 9.01s
165:	learn: 41400.3076281	total: 1.79s	remaining: 8.98s
166:	learn: 41369.2945458	total: 1.79s	remaining: 8.94s
167:	learn: 41364.8782141	total: 1.8s	remaining: 8.9s
168:	learn: 41327.0890259	total: 1.81s	remaining: 8.88s
169:	learn: 41299.7132478	total: 1.81s	remaining: 8.85s
170:	learn: 41276.5820775	total: 1.82s	remaining: 8

307:	learn: 38049.0899942	total: 2.82s	remaining: 6.33s
308:	learn: 38012.5425367	total: 2.83s	remaining: 6.32s
309:	learn: 37980.1424681	total: 2.83s	remaining: 6.3s
310:	learn: 37957.3689399	total: 2.84s	remaining: 6.29s
311:	learn: 37926.3569075	total: 2.84s	remaining: 6.27s
312:	learn: 37904.1653681	total: 2.85s	remaining: 6.25s
313:	learn: 37877.8398438	total: 2.85s	remaining: 6.23s
314:	learn: 37849.9212452	total: 2.86s	remaining: 6.22s
315:	learn: 37831.4314751	total: 2.87s	remaining: 6.2s
316:	learn: 37793.5814518	total: 2.87s	remaining: 6.18s
317:	learn: 37756.7466508	total: 2.88s	remaining: 6.17s
318:	learn: 37755.5462977	total: 2.88s	remaining: 6.15s
319:	learn: 37734.3461082	total: 2.89s	remaining: 6.14s
320:	learn: 37712.1688325	total: 2.89s	remaining: 6.12s
321:	learn: 37678.8419215	total: 2.9s	remaining: 6.11s
322:	learn: 37659.4288033	total: 2.91s	remaining: 6.09s
323:	learn: 37625.7492710	total: 2.91s	remaining: 6.08s
324:	learn: 37605.3623692	total: 2.92s	remaining: 6

474:	learn: 34843.3651233	total: 4.21s	remaining: 4.66s
475:	learn: 34832.1626539	total: 4.22s	remaining: 4.65s
476:	learn: 34807.4757345	total: 4.24s	remaining: 4.64s
477:	learn: 34787.0076949	total: 4.24s	remaining: 4.63s
478:	learn: 34767.2201228	total: 4.25s	remaining: 4.63s
479:	learn: 34752.6302409	total: 4.26s	remaining: 4.62s
480:	learn: 34746.2939157	total: 4.28s	remaining: 4.61s
481:	learn: 34729.2672036	total: 4.29s	remaining: 4.61s
482:	learn: 34711.6089584	total: 4.29s	remaining: 4.6s
483:	learn: 34696.4682801	total: 4.3s	remaining: 4.59s
484:	learn: 34680.2374823	total: 4.32s	remaining: 4.58s
485:	learn: 34665.4566434	total: 4.33s	remaining: 4.58s
486:	learn: 34644.0629828	total: 4.34s	remaining: 4.57s
487:	learn: 34618.1531997	total: 4.35s	remaining: 4.56s
488:	learn: 34606.3220492	total: 4.36s	remaining: 4.55s
489:	learn: 34593.4050462	total: 4.37s	remaining: 4.54s
490:	learn: 34581.6396846	total: 4.38s	remaining: 4.54s
491:	learn: 34569.4611838	total: 4.39s	remaining: 

625:	learn: 32658.1548887	total: 6.29s	remaining: 3.76s
626:	learn: 32645.4327985	total: 6.31s	remaining: 3.75s
627:	learn: 32635.7594596	total: 6.34s	remaining: 3.76s
628:	learn: 32625.3483879	total: 6.37s	remaining: 3.76s
629:	learn: 32610.4518253	total: 6.39s	remaining: 3.76s
630:	learn: 32594.0150189	total: 6.42s	remaining: 3.75s
631:	learn: 32579.9359798	total: 6.44s	remaining: 3.75s
632:	learn: 32564.8808937	total: 6.45s	remaining: 3.74s
633:	learn: 32560.8957823	total: 6.47s	remaining: 3.73s
634:	learn: 32545.1947570	total: 6.48s	remaining: 3.73s
635:	learn: 32533.9123233	total: 6.5s	remaining: 3.72s
636:	learn: 32525.0138766	total: 6.52s	remaining: 3.71s
637:	learn: 32509.7082288	total: 6.55s	remaining: 3.72s
638:	learn: 32501.4291219	total: 6.57s	remaining: 3.71s
639:	learn: 32486.3885057	total: 6.59s	remaining: 3.71s
640:	learn: 32467.7945811	total: 6.61s	remaining: 3.7s
641:	learn: 32453.3940459	total: 6.62s	remaining: 3.69s
642:	learn: 32432.6996908	total: 6.66s	remaining: 

...

325:	learn: 38048.3428160	total: 2.36s	remaining: 4.89s
326:	learn: 38027.4465722	total: 2.37s	remaining: 4.88s
327:	learn: 38018.3930104	total: 2.38s	remaining: 4.87s
328:	learn: 38002.7794297	total: 2.38s	remaining: 4.86s
329:	learn: 37985.4556866	total: 2.39s	remaining: 4.86s
330:	learn: 37958.8555940	total: 2.4s	remaining: 4.85s
331:	learn: 37938.4987371	total: 2.41s	remaining: 4.84s
332:	learn: 37936.0289171	total: 2.41s	remaining: 4.83s
333:	learn: 37923.0428612	total: 2.42s	remaining: 4.82s
334:	learn: 37911.4273134	total: 2.43s	remaining: 4.82s
335:	learn: 37896.4967349	total: 2.43s	remaining: 4.81s
336:	learn: 37871.5372519	total: 2.44s	remaining: 4.8s
337:	learn: 37845.1825479	total: 2.45s	remaining: 4.8s
338:	learn: 37833.6463105	total: 2.46s	remaining: 4.79s
339:	learn: 37784.9775868	total: 2.46s	remaining: 4.78s
340:	learn: 37768.0525632	total: 2.47s	remaining: 4.77s
341:	learn: 37733.0879416	total: 2.48s	remaining: 4.76s
342:	learn: 37721.9829399	total: 2.48s	remaining: 4

487:	learn: 35320.6322747	total: 3.54s	remaining: 3.71s
488:	learn: 35309.4062234	total: 3.55s	remaining: 3.71s
489:	learn: 35293.1038571	total: 3.55s	remaining: 3.7s
490:	learn: 35280.8538126	total: 3.56s	remaining: 3.69s
491:	learn: 35270.3905381	total: 3.57s	remaining: 3.69s
492:	learn: 35265.1089128	total: 3.58s	remaining: 3.68s
493:	learn: 35260.1748697	total: 3.58s	remaining: 3.67s
494:	learn: 35245.9784787	total: 3.59s	remaining: 3.66s
495:	learn: 35225.6306125	total: 3.6s	remaining: 3.65s
496:	learn: 35213.0041321	total: 3.6s	remaining: 3.65s
497:	learn: 35189.4618168	total: 3.61s	remaining: 3.64s
498:	learn: 35186.5533500	total: 3.62s	remaining: 3.63s
499:	learn: 35186.0189752	total: 3.62s	remaining: 3.62s
500:	learn: 35182.7005865	total: 3.63s	remaining: 3.61s
501:	learn: 35169.7016560	total: 3.63s	remaining: 3.6s
502:	learn: 35167.5586950	total: 3.64s	remaining: 3.6s
503:	learn: 35152.4862541	total: 3.65s	remaining: 3.59s
504:	learn: 35136.7550114	total: 3.65s	remaining: 3.5

653:	learn: 33496.0104250	total: 4.72s	remaining: 2.5s
654:	learn: 33483.1208525	total: 4.73s	remaining: 2.49s
655:	learn: 33480.5565438	total: 4.74s	remaining: 2.48s
656:	learn: 33471.2791451	total: 4.74s	remaining: 2.48s
657:	learn: 33449.4749918	total: 4.75s	remaining: 2.47s
658:	learn: 33437.8026072	total: 4.76s	remaining: 2.46s
659:	learn: 33427.3089731	total: 4.76s	remaining: 2.46s
660:	learn: 33403.2889048	total: 4.77s	remaining: 2.45s
661:	learn: 33397.1269369	total: 4.78s	remaining: 2.44s
662:	learn: 33379.6331323	total: 4.79s	remaining: 2.43s
663:	learn: 33372.6706302	total: 4.79s	remaining: 2.42s
664:	learn: 33360.8923225	total: 4.8s	remaining: 2.42s
665:	learn: 33351.3214463	total: 4.81s	remaining: 2.41s
666:	learn: 33337.3458135	total: 4.81s	remaining: 2.4s
667:	learn: 33330.2614248	total: 4.82s	remaining: 2.4s
668:	learn: 33309.3648611	total: 4.83s	remaining: 2.39s
669:	learn: 33305.3900438	total: 4.83s	remaining: 2.38s
670:	learn: 33293.2757438	total: 4.84s	remaining: 2.

819:	learn: 31721.0557686	total: 5.9s	remaining: 1.29s
820:	learn: 31706.1741011	total: 5.91s	remaining: 1.29s
821:	learn: 31694.8742427	total: 5.92s	remaining: 1.28s
822:	learn: 31687.5671476	total: 5.92s	remaining: 1.27s
823:	learn: 31687.2943842	total: 5.93s	remaining: 1.27s
824:	learn: 31675.4079592	total: 5.94s	remaining: 1.26s
825:	learn: 31662.0010341	total: 5.94s	remaining: 1.25s
826:	learn: 31652.3809632	total: 5.95s	remaining: 1.24s
827:	learn: 31644.3296912	total: 5.96s	remaining: 1.24s
828:	learn: 31644.0993740	total: 5.96s	remaining: 1.23s
829:	learn: 31628.7233063	total: 5.97s	remaining: 1.22s
830:	learn: 31622.9143879	total: 5.98s	remaining: 1.22s
831:	learn: 31611.0572168	total: 5.98s	remaining: 1.21s
832:	learn: 31590.3599811	total: 5.99s	remaining: 1.2s
833:	learn: 31590.1399087	total: 6s	remaining: 1.19s
834:	learn: 31577.1584679	total: 6s	remaining: 1.19s
835:	learn: 31575.3372449	total: 6.01s	remaining: 1.18s
836:	learn: 31568.2720431	total: 6.02s	remaining: 1.17s


985:	learn: 30235.4486340	total: 7.08s	remaining: 101ms
986:	learn: 30232.4198920	total: 7.09s	remaining: 93.4ms
987:	learn: 30222.9541375	total: 7.1s	remaining: 86.2ms
988:	learn: 30214.7065027	total: 7.11s	remaining: 79ms
989:	learn: 30208.9609444	total: 7.12s	remaining: 71.9ms
990:	learn: 30202.1102911	total: 7.12s	remaining: 64.7ms
991:	learn: 30192.4166085	total: 7.13s	remaining: 57.5ms
992:	learn: 30183.2839628	total: 7.14s	remaining: 50.3ms
993:	learn: 30180.8327715	total: 7.14s	remaining: 43.1ms
994:	learn: 30173.7294191	total: 7.15s	remaining: 35.9ms
995:	learn: 30166.8658446	total: 7.16s	remaining: 28.8ms
996:	learn: 30161.7494584	total: 7.17s	remaining: 21.6ms
997:	learn: 30149.5842452	total: 7.17s	remaining: 14.4ms
998:	learn: 30148.3347678	total: 7.18s	remaining: 7.19ms
999:	learn: 30143.4242024	total: 7.19s	remaining: 0us


In [1489]:
r2_score(y,y_pred), scores.sum()/5

(0.9328742334490399, 0.7676266682381907)

In [1490]:
preds_final = pd.DataFrame()
preds_final['Id'] = X_target['Id'].copy()
y_pred_final = model.predict(X_target)

preds_final['Price'] = y_pred_final
preds_final.to_csv('predictions.csv', index=False)